### making site_sandali excel files in site folder(bahar_sandali_All + electro is needed)
### use_bahar_sandali_full = bahar_sandali_All + electro

In [22]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# مسیر فایل ورودی
file_path = 'bahar_sandali_full.xlsx'

# خواندن فایل اکسل
df = pd.read_excel(file_path)

# ساخت پوشه برای ذخیره فایل‌های خروجی
output_dir = 'site_excell'
os.makedirs(output_dir, exist_ok=True)

# دیکشنری عرض ستون‌ها (برحسب character width)
column_widths = {
    'std': 15,
    'name': 10,
    'fname': 13,
    'code': 10,       # ≈ 100 pixels
    'dars': 30,       # ≈ 300 pixels
    'tarikh': 15,
    'saat': 12,
    'sandali': 8,
    'noe': 10,
    'makan': 15
}

# گروه‌بندی بر اساس tarikh و saat
grouped = df.groupby(['tarikh', 'saat'])

for (tarikh, saat), group in grouped:
    # مرتب‌سازی بر اساس ستون fname
    sorted_group = group.sort_values(by='fname')

    # ساخت نام فایل
    filename = f"{tarikh}_{saat}".replace("/", "-").replace(":", "-") + ".xlsx"
    output_path = os.path.join(output_dir, filename)

    # ذخیره به اکسل
    sorted_group.to_excel(output_path, index=False)

    # اعمال تنظیمات ظاهری با openpyxl
    wb = load_workbook(output_path)
    ws = wb.active

    # وسط‌چین کردن همه سلول‌ها و تنظیم عرض ستون‌ها
    for col_idx, column_cells in enumerate(ws.columns, start=1):
        col_letter = get_column_letter(col_idx)
        header = ws[f'{col_letter}1'].value
        if header in column_widths:
            ws.column_dimensions[col_letter].width = column_widths[header]
        for cell in column_cells:
            cell.alignment = Alignment(horizontal='center', vertical='center')

    wb.save(output_path)
print("Excell files created exesfuliy")



Excell files created exesfuliy


###  make word files

In [ ]:
import pandas as pd
import os
from docx import Document
from docx.shared import Pt, Inches
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH

def set_repeat_table_header(row):
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    tblHeader = OxmlElement('w:tblHeader')
    trPr.append(tblHeader)

def set_cell_background(cell, color_hex):
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    shd = OxmlElement('w:shd')
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    shd.set(qn('w:fill'), color_hex)
    tcPr.append(shd)

def prevent_row_split(row):
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    cant_split = OxmlElement('w:cantSplit')
    trPr.append(cant_split)

def set_cell_alignment_right(cell):
    for paragraph in cell.paragraphs:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# فایل اکسل
file_path = 'bahar_sandali_full.xlsx'
output_dir = 'site_word'
os.makedirs(output_dir, exist_ok=True)

column_map = {
    'std': 'شماره دانشجویی',
    'fname': 'نام خانوادگی',
    'name': 'نام',
    'code': 'کد درس',
    'dars': 'نام درس',
    'tarikh': 'تاریخ',
    'saat': 'ساعت',
    'sandali': 'شماره صندلی',
    'noe': 'نوع امتحان',
    'makan': 'مکان',
}

display_columns = [c for c in column_map if c not in ['tarikh', 'saat']]

column_widths_px = {
    'std': 100,
    'fname': 110,
    'name': 60,
    'code': 80,
    'dars': 360,
    'sandali': 65,
    'noe': 100,
    'makan': 210
}
column_widths_inch = {k: v / 96 for k, v in column_widths_px.items()}

df = pd.read_excel(file_path)
df = df[[c for c in column_map.keys()]]

grouped = df.groupby(['tarikh', 'saat'])

for (tarikh, saat), group in grouped:
    group = group.sort_values(by='fname')
    doc = Document()

    # فونت و راست‌چین پیش‌فرض
    style = doc.styles['Normal']
    style.font.name = 'B Nazanin'
    style._element.rPr.rFonts.set(qn('w:eastAsia'), 'B Nazanin')
    style.font.size = Pt(10)

    # جدول
    table = doc.add_table(rows=1, cols=len(display_columns))
    table.style = 'Table Grid'
    table.alignment = WD_TABLE_ALIGNMENT.RIGHT

    # ردیف هدر
    header_row = table.rows[0]
    for i, col in enumerate(display_columns):
        cell = header_row.cells[i]
        cell.text = column_map[col]
        set_cell_background(cell, 'D9D9D9')
        set_cell_alignment_right(cell)
        cell.width = Inches(column_widths_inch.get(col, 1.2))
    set_repeat_table_header(header_row)
    prevent_row_split(header_row)

    # ردیف‌های اطلاعاتی
    for idx, (_, row_data) in enumerate(group.iterrows()):
        row = table.add_row()
        prevent_row_split(row)
        bg_color = 'FFFFFF' if idx % 2 == 0 else 'F2F2F2'
        for i, col in enumerate(display_columns):
            cell = row.cells[i]
            cell.text = str(row_data[col])
            set_cell_background(cell, bg_color)
            set_cell_alignment_right(cell)
            cell.width = Inches(column_widths_inch.get(col, 1.2))

    # ردیف آخر خالی
    row = table.add_row()
    prevent_row_split(row)
    for cell in row.cells:
        cell.text = ""
        set_cell_alignment_right(cell)

    # ذخیره فایل
    safe_name = f"{tarikh}_{saat}".replace("/", "-").replace(":", "-")
    docx_path = os.path.join(output_dir, f"{safe_name}.docx")
    if os.path.exists(docx_path):
        os.remove(docx_path)
    doc.save(docx_path)
print("word files created successfully")


The word files created successfully


### convert to pdf

In [25]:
import os
from docx2pdf import convert

# مسیر ورودی و خروجی
input_dir = 'site_word'
output_dir = 'site_pdf'

# ساخت پوشه خروجی در صورت نبود
os.makedirs(output_dir, exist_ok=True)

# پیمایش فایل‌ها در پوشه site
for filename in os.listdir(input_dir):
    if filename.endswith('.docx'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.docx', '.pdf'))
        convert(input_path, output_path)

print("تبدیل همه فایل‌های Word به PDF با موفقیت انجام شد.")

100%|██████████| 1/1 [00:16<00:00, 16.58s/it]

تبدیل همه فایل‌های Word به PDF با موفقیت انجام شد.
